In [1]:
!git clone https://github.com/leeskelton8383/EaglesGPT.git
%cd EaglesGPT
import os
REPO_DIR = "/content/YOUR_REPO_NAME"   # <-- replace with your repo folder name
os.chdir(REPO_DIR)
print("Now working in:", os.getcwd())


Cloning into 'EaglesGPT'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 36 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 2.52 MiB | 5.59 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/EaglesGPT


1. Load and Chunk the wikipedia narrative docs.

In [3]:
import os
import json


# -----------------------------------
# Step 1: Chunking Function
# -----------------------------------
def chunk_text_fixed(text, chunk_size=300, overlap=50):
    """
    Splits text into overlapping chunks.
    Each chunk has `chunk_size` words, with `overlap` words shared with the next chunk.
    """
    words = text.split()
    chunks = []
    step = chunk_size - overlap

    if step <= 0:
        raise ValueError("chunk_size must be greater than overlap")

    for i in range(0, len(words), step):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)

    return chunks

# -----------------------------------
# Step 2: Load .txt Files, Chunk, Print Examples, Save as JSONL
# -----------------------------------
def load_and_chunk_wikipedia(folder_path, output_path, chunk_size=300, overlap=50):
    all_chunks = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            full_path = os.path.join(folder_path, filename)

            with open(full_path, 'r', encoding='utf-8') as f:
                text = f.read()
                doc_title = filename.replace('.txt', '')

                chunks = chunk_text_fixed(text, chunk_size=chunk_size, overlap=overlap)

                for i, chunk in enumerate(chunks):
                    chunk_id = f"{doc_title}_{str(i+1).zfill(3)}"
                    all_chunks.append({
                        'chunk_id': chunk_id,
                        'doc_title': doc_title,
                        'text': chunk
                    })

    # --- Print first 3 chunks for inspection ---
    print("\n--- Example Chunks (JSON) ---")
    for chunk in all_chunks[:3]:
        print(json.dumps(chunk, indent=2))

    # --- Save all chunks to JSONL ---
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as out_file:
        for chunk in all_chunks:
            out_file.write(json.dumps(chunk) + '\n')

    print(f"\n✅ Saved {len(all_chunks)} chunks to {output_path}")
    return all_chunks

# -----------------------------------
# Step 3: Setup Clean Project Paths (portable)
# -----------------------------------

PROJECT_ROOT = os.getcwd()
NARRATIVE_DIR = os.path.join(PROJECT_ROOT, 'data', 'narratives')
CHUNKS_OUT_PATH = os.path.join(PROJECT_ROOT, 'data', 'chunks.jsonl')

# --- Run It ---
chunks = load_and_chunk_wikipedia(folder_path=NARRATIVE_DIR, output_path=CHUNKS_OUT_PATH)



--- Example Chunks (JSON) ---
{
  "chunk_id": "2022_philadelphia_eagles_season_001",
  "doc_title": "2022_philadelphia_eagles_season",
  "text": "The 2022 season was the Philadelphia Eagles' 90th in the National Football League (NFL), their 20th playing home games at Lincoln Financial Field, their second under head coach Nick Sirianni, and seventh under general manager Howie Roseman since he returned to the position (twelfth overall). The Eagles started 8\u20130 for the first time in franchise history, before their winning streak was snapped with a 32\u201321 upset loss to the Washington Commanders in Week 10. They improved on their 9\u20138 record from last year after a 40\u201333 victory over the Green Bay Packers in Week 12. With a 48\u201322 road win over the division rival New York Giants in Week 14, the Eagles clinched their second straight playoff berth, and their fifth in six seasons. After a 25\u201320 road win over the Chicago Bears in Week 15, the Eagles matched their franc

2. Embed the corpus and index

In [ ]:
! pip install -U sentence-transformers
! pip install faiss-cpu
! pip install -U bitsandbytes
! pip install --upgrade openai

import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
from openai import OpenAI
import re

# -----------------------------------
# Path Setup (works in Colab or local)
# -----------------------------------
NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR))  # assuming /content/EaglesGPT in Colab

CHUNKS_JSONL_PATH = os.path.join(PROJECT_ROOT, 'data', 'chunks.jsonl')
FAISS_INDEX_PATH = os.path.join(PROJECT_ROOT, 'data', 'narrative_index.faiss')
METADATA_JSON_PATH = os.path.join(PROJECT_ROOT, 'data', 'narrative_metadata.json')

# -----------------------------------
# Load Chunks
# -----------------------------------
def load_chunks_from_jsonl(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

# -----------------------------------
# Embed Chunks
# -----------------------------------
def embed_chunks(chunks, model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    texts = [chunk['text'] for chunk in chunks]
    print(f"🔁 Embedding {len(texts)} chunks...")
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=32)
    for i, emb in enumerate(embeddings):
        chunks[i]['embedding'] = emb.tolist()
    return chunks

# -----------------------------------
# Build FAISS Index
# -----------------------------------
def build_faiss_index(embedded_chunks, index_path, metadata_path):
    embeddings = np.array([c['embedding'] for c in embedded_chunks]).astype('float32')
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    os.makedirs(os.path.dirname(index_path), exist_ok=True)
    faiss.write_index(index, index_path)
    print(f"✅ FAISS index saved to {index_path}")
    metadata = [{k: c[k] for k in ['chunk_id', 'doc_title', 'text']} for c in embedded_chunks]
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2)
    print(f"✅ Metadata saved to {metadata_path}")

# -----------------------------------
# Retrieve Chunks
# -----------------------------------

def extract_years(text):
    return re.findall(r'20\d{2}', text)

def retrieve_narrative_chunks(question, index_path, metadata_path, model_name='all-MiniLM-L6-v2', top_k=5):
    print(f"🔍 Retrieving top {top_k} chunks for question: {question}")

    # Step 1: Extract years from question
    target_years = set(extract_years(question))

    # Step 2: Load index + metadata
    index = faiss.read_index(index_path)
    with open(metadata_path, 'r', encoding='utf-8') as f:
        metadata = json.load(f)

    # Step 3: Embed the question
    model = SentenceTransformer(model_name)
    query_vector = model.encode([question]).astype('float32')

    # Step 4: Search index
    distances, indices = index.search(query_vector, top_k * 3)  # get more than needed
    candidates = [metadata[i] for i in indices[0]]

    # Step 5: Re-rank to prioritize matches to target years
    def score(chunk):
        title = chunk['doc_title']
        if any(y in title for y in target_years):
            return 0  # best score
        return 1  # deprioritize

    candidates.sort(key=score)
    results = candidates[:top_k]

    # Print results
    print(f"\n📄 Top Retrieved Chunks (boosted by years: {', '.join(target_years) if target_years else 'None'}):\n")
    for i, r in enumerate(results):
        print(f"[{i+1}] ({r['chunk_id']}) from {r['doc_title']}")
        print(r['text'][:500] + "\n---\n")

    return results
# -----------------------------------
# Use openai to Answer
# -----------------------------------


# ✅ Set API key (do this once in the notebook)
os.environ["OPENAI_API_KEY"] = "sk-"#YOUR_API_KEY_HERE
# ✅ Create client
client = OpenAI()

def answer_with_openai(question, retrieved_chunks, model="gpt-3.5-turbo"):
    context = "\n\n".join(chunk["text"] for chunk in retrieved_chunks)

    system_prompt = (
        "You are a knowledgeable assistant helping summarize NFL team narratives. "
        "Use only the context provided. Do not hallucinate or include outside information."
    )

    user_prompt = f"""Answer the question based on the following Eagles season narratives:

{context}

Question: {question}
Answer:"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2
    )

    answer = response.choices[0].message.content
    print("\n🤖 OpenAI's Answer:\n")
    print(answer)
    return answer

In [25]:


question = "which was more disappointing the 2020 or 2023 season"
retrieved = retrieve_narrative_chunks(question, FAISS_INDEX_PATH, METADATA_JSON_PATH)
answer= answer_with_openai(question,retrieved)

🔍 Retrieving top 5 chunks for question: which was more disappointing the 2020 or 2023 season

📄 Top Retrieved Chunks (boosted by years: 2023, 2020):

[1] (2020_philadelphia_eagles_season_001) from 2020_philadelphia_eagles_season
The 2020 season was the Philadelphia Eagles' 88th in the National Football League (NFL) and their fifth and final under head coach Doug Pederson. They failed to improve on their 9–7 record from the previous season following a 23–17 loss to the Seattle Seahawks in Week 12. They were eliminated from playoff contention for the first time since 2016 following a Week 16 loss to the Dallas Cowboys and finished with a dismal 4–11–1 record, the second-worst in the National Football Conference (NFC), and
---

[2] (2020_philadelphia_eagles_season_002) from 2020_philadelphia_eagles_season
home games would have no fans in attendance. However, starting in week 6, Philadelphia Mayor Jim Kenney announced that the city would allow 7,500 fans to attend Eagles home games. This w